# Pyvista + TimeSeriesStorage

Want to find a nice way to work with time-dependent data visualization in a jupyter notebook. Will be using a pyvistaqt BackgroundPlotter, which should read data directly from and hdf5 file with all time-dependent data.

In [4]:
from time import sleep

In [10]:
def visualize_timedependent(hdf5file, pl: BackgroundPlotter):
    time = read_time(hdf5file)
    mesh = read_mesh(hdf5file)
    u = read_function(hf5file, name, 0.0)
    
    pyvis(mesh)
    pl.add_mesh()
    
    timer = PyvisTimer()
    for ti in time:
        timer.wait(ti)
        update_scalar(u, pl)
        update_text(ti, pl)

NameError: name 'BackgroundPlotter' is not defined

In [11]:
from dolfin import *
import pyvista as pv
import pyvistaqt as pvqt
from pathlib import Path
import numpy as np

In [12]:
from dolfin import *
import pyvista as pv
import pyvistaqt as pvqt
from pathlib import Path
import numpy as np

path = Path("../pyvistualizer").resolve() / "pyvistualier.h5"
mesh = RectangleMesh(Point(-1, -1), Point(1, 1), 10, 10)
V = FunctionSpace(mesh, "CG", 1)
el = V.element()
sig = el.signature() 
t = Constant(0.)
u = Expression(
    "exp(-(pow(x[0], 2) + pow(x[1], 2)) / (0.25 * (1.0+t*t))) /(1.0+t*t)",
    t=t, degree=1
)
u.rename("u", "")
dt = 0.1

In [ ]:
def encode_signature(signature: str, mpiwrapper):
    arr = np.array(bytearray(signature.encode()))
    petv = PETScVector(mpiwrapper, arr.size)
    petv[:] = arr
    return petv

def decode_signature(petv: PETScVector):
    return bytearray(petv[:].astype(np.uint8)).decode()

In [3]:
with HDF5File(mesh.mpi_comm(), str(path), "w") as f:
    f.write(encode_signature(sig, mesh.mpi_comm()), '/element')
    f.write(mesh, "/mesh")
    name = u.name()
    for idx, _ in enumerate(range(10)):
        uh = project(u, V)
        t.assign(t+dt)
        f.write(uh.vector(), f"/{name}/{name}_{idx}")

In [4]:
mesh = Mesh()
petv = PETScVector(mesh.mpi_comm())
with HDF5File(mesh.mpi_comm(), str(path), "r") as f:
    f.read(mesh, "/mesh", True)
    f.read(petv, "/element", True)
    el = eval(decode_signature(petv))
    V = FunctionSpace(mesh, el)
    u0 = Function(V)
    f.read(u0.vector(), f"/{u.name()}/{u.name()}_5", True)
    

In [5]:
import matplotlib.pyplot as plt

In [13]:
c = plot(u0, vmax=1)
plt.colorbar(c)

NameError: name 'u0' is not defined

In [58]:
def find_nearest(array,value):
    # Might be useful for future timeline widghet
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

In [59]:
import pyvista
pyvista.set_jupyter_backend("ipygany")
grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(V))
def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter()
    # Update point values on pyvista grid
    grid.point_data[f"u({t})"] = uh.x.array.real
    # Warp mesh by point values
    warped = grid.warp_by_scalar(f"u({t})", factor=1.5)

    # Add mesh to plotter and visualize in notebook or save as figure
    actor = p.add_mesh(warped)
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
        pyvista.start_xvfb()
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)
plot_function(0, uh)

AttributeError: 'function' object has no attribute 'create_vtk_mesh'